In [206]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import datetime

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# 最大表示列数の指定（ここでは50列を指定）
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)

In [207]:
path = "../input/"

df_category_names = pd.read_csv(path + 'category_names.csv')
df_item_categories = pd.read_csv(path + 'item_categories.csv')
df_sample_submission = pd.read_csv(path + 'sample_submission.csv')
df_sales_history = pd.read_csv(path + 'sales_history.csv')

test = pd.read_csv(path + 'test.csv')
test = test.drop(["index"], axis=1)

In [208]:
df_sales_history

,日付,店舗ID,商品ID,商品価格,売上個数
0,2018-01-01,9,1000001,420,1.0
1,2018-01-01,6,1000001,420,1.0
2,2018-01-01,10,1000001,420,1.0
3,2018-01-01,0,1000017,250,1.0
4,2018-01-01,1,1000017,250,1.0
...,...,...,...,...,...
1119565,2019-10-31,8,3400006,2700,1.0
1119566,2019-10-31,7,3400009,1020,1.0
1119567,2019-10-31,7,3400012,510,1.0
1119568,2019-10-31,4,3500001,340,1.0


<br>
<p>カテゴリーデータの情報付与</p>

In [209]:
# まず、カテゴリー名称の情報をアイテムリストに情報を付与する
print(f"マージ前{len(df_item_categories)}")
df_item_categories = pd.merge(df_item_categories, df_category_names, on="商品カテゴリID", how="inner")
print(f"マージ後{len(df_item_categories)}")

# カテゴリー名から大分類の情報を抽出
df_item_categories["商品カテゴリ名_大分類"] = df_item_categories["商品カテゴリ名"].str.extract('(.+) -', expand=True)

# カテゴリーの情報をdf_sales_historyとtestに付与
print(f"マージ前{len(df_sales_history)}")
df_sales_history = pd.merge(df_sales_history, df_item_categories ,on="商品ID", how="inner")
print(f"マージ後{len(df_sales_history)}")

print(f"マージ前{len(test)}")
test = pd.merge(test, df_item_categories ,on="商品ID", how="inner")
print(f"マージ後{len(test)}")

マージ前9426
マージ後9426
マージ前1119570
マージ後1119570
マージ前3060
マージ後3060


<p>時系列データを使用して特徴量を作成する。</p>

In [210]:
df_sales_history["日付"] = pd.to_datetime(df_sales_history["日付"], format='%Y-%m-%d')
df_sales_history["年月"] = df_sales_history["日付"].dt.strftime('%Y-%m')
df_sales_history["年"] = df_sales_history["日付"].dt.year
df_sales_history["月"] = df_sales_history["日付"].dt.month
df_sales_history["日"] = df_sales_history["日付"].dt.day

test["月"] = 12
test["年"] = 2019
test["日付"] = pd.to_datetime("2019-12-15")

# 最新の価格を付与するために最後に売れた日の価格データを取得
item_final_price = df_sales_history.groupby(["店舗ID","商品ID"], as_index=False)["年月"].max()

# 同日中に価格が変化している商品あり、最小値と最大値のデータを付与
df_sales_history["item_price_max"] = df_sales_history["商品価格"]
df_sales_history["item_price_min"] = df_sales_history["商品価格"]
df_sales_history["item_price_mean"] = df_sales_history["商品価格"]

tmp = df_sales_history.groupby(["年月", "店舗ID","商品ID", "item_price_max", "item_price_min"], as_index=False)["売上個数"].sum()
tmp = df_sales_history.groupby(["年月", "店舗ID","商品ID"], as_index=False).agg({"item_price_max":"max","item_price_min":"min","item_price_mean":"mean"})

print(f"マージ前{len(item_final_price)}")
item_final_price = pd.merge(item_final_price, tmp[["商品ID", "店舗ID", "年月", "item_price_max", "item_price_min", "item_price_mean"]],
                            on=["商品ID", "店舗ID", "年月"], how="inner")
print(f"マージ前{len(item_final_price)}")

print(f"マージ前{len(test)}")
test = pd.merge(test, item_final_price[["商品ID", "店舗ID", "item_price_max", "item_price_min", "item_price_mean"]], on=["商品ID", "店舗ID"], how="left")
print(f"マージ前{len(test)}")

# 去年のカテゴリー毎の商品価格の中央値を取る
tmp = df_sales_history[df_sales_history["月"]==12].groupby(["商品カテゴリID"], as_index=False)["商品価格"].median().rename(columns={"商品価格":"カテゴリー価格"})

print(f"マージ前{len(test)}")
test = pd.merge(test, tmp, on=["商品カテゴリID"], how="left")
print(f"マージ前{len(test)}")

test["item_price_max"] = test["item_price_max"].fillna(test["カテゴリー価格"])
test["item_price_min"] = test["item_price_min"].fillna(test["カテゴリー価格"])
test["item_price_mean"] = test["item_price_mean"].fillna(test["カテゴリー価格"])

test.drop(["カテゴリー価格", "商品カテゴリ名"], axis=1, inplace=True)

マージ前107115
マージ前107115
マージ前3060
マージ前3060
マージ前3060
マージ前3060


In [211]:
# nullがないことを確認
test.isnull().sum()

商品ID               0
店舗ID               0
商品カテゴリID           0
商品カテゴリ名_大分類        0
月                  0
年                  0
日付                 0
item_price_max     0
item_price_min     0
item_price_mean    0
dtype: int64

<br>
<p>発売してからの経過日を出す。</p>

In [212]:
item_release_date_list = df_sales_history.groupby("商品ID", as_index=False)["日付"].min().rename(columns={"日付":"発売日"})

print(f"マージ前{len(df_sales_history)}")
df_sales_history = pd.merge(df_sales_history, item_release_date_list, on="商品ID", how="inner")
print(f"マージ後{len(df_sales_history)}")

print(f"マージ前{len(test)}")
test = pd.merge(test, item_release_date_list, on="商品ID", how="left")
print(f"マージ後{len(test)}")

# 発売していない商品は12/15で設定
test["発売日"] = test["発売日"].fillna(pd.to_datetime(2019-12-15))

df_sales_history["発売後経過期間"] = df_sales_history["日付"] - df_sales_history["発売日"]

test["発売後経過期間"] = test["日付"] - test["発売日"] 

test.drop("発売日", axis=1, inplace=True)

マージ前1119570
マージ後1119570
マージ前3060
マージ後3060


In [213]:
df_sales_history["発売後経過期間"] = df_sales_history["発売後経過期間"].astype(str)
df_sales_history["発売後経過期間"] = df_sales_history["発売後経過期間"].str.extract('(.+) d', expand=True)
df_sales_history["発売後経過期間"] = df_sales_history["発売後経過期間"].astype(int)

test["発売後経過期間"] = test["発売後経過期間"].astype(str)
test["発売後経過期間"] = test["発売後経過期間"].str.extract('(.+) d', expand=True)
test["発売後経過期間"] = test["発売後経過期間"].astype(int)

In [214]:
test.head()

,商品ID,店舗ID,商品カテゴリID,商品カテゴリ名_大分類,月,年,日付,item_price_max,item_price_min,item_price_mean,発売後経過期間
0,1000001,0,100,映画,12,2019,2019-12-15,250.0,250.0,250.0,713
1,1000001,1,100,映画,12,2019,2019-12-15,420.0,420.0,420.0,713
2,1000001,2,100,映画,12,2019,2019-12-15,250.0,250.0,250.0,713
3,1000001,3,100,映画,12,2019,2019-12-15,250.0,250.0,250.0,713
4,1000001,4,100,映画,12,2019,2019-12-15,250.0,250.0,250.0,713


<br>
<p>月ごとのデータに変換する。</p>

In [215]:
train = df_sales_history.groupby(["年", "月", "店舗ID", "商品ID", "商品カテゴリID", "商品カテゴリ名_大分類"], as_index=False).\
        agg({"発売後経過期間":"mean","売上個数":"sum","item_price_max":"max", "item_price_min":"min", "item_price_mean":"mean"})

# train = df_sales_history.groupby(["年", "月", "店舗ID", "商品ID", "商品カテゴリID", "商品カテゴリ名_大分類", "item_price_mean"], as_index=False).\
#         agg({"発売後経過期間":"mean","売上個数":"sum","item_price_max":"max", "item_price_min":"min"})

train

,年,月,店舗ID,商品ID,商品カテゴリID,商品カテゴリ名_大分類,発売後経過期間,売上個数,item_price_max,item_price_min,item_price_mean
0,2018,1,0,1000001,100,映画,12.600,6.0,420,420,420.0
1,2018,1,0,1000002,100,映画,10.000,2.0,250,250,250.0
2,2018,1,0,1000004,100,映画,15.000,1.0,120,120,120.0
3,2018,1,0,1000005,100,映画,11.875,8.0,250,130,220.0
4,2018,1,0,1000006,100,映画,4.000,2.0,250,130,190.0
...,...,...,...,...,...,...,...,...,...,...,...
495129,2019,10,17,3400005,340,ゲーム周辺機器,661.000,1.0,1190,1190,1190.0
495130,2019,10,17,3400006,340,ゲーム周辺機器,642.000,1.0,2700,2700,2700.0
495131,2019,10,17,3400012,340,ゲーム周辺機器,639.000,1.0,680,680,680.0
495132,2019,10,17,3400016,340,ゲーム周辺機器,141.000,1.0,250,250,250.0


In [216]:
df_sales_history[df_sales_history["商品カテゴリ名_大分類"]=="ギフト"]

,日付,店舗ID,商品ID,商品価格,売上個数,商品カテゴリID,商品カテゴリ名,商品カテゴリ名_大分類,年月,年,月,日,item_price_max,item_price_min,item_price_mean,発売日,発売後経過期間
160200,2018-01-01,10,1600002,510,1.0,160,ギフト - 小型ゲーム,ギフト,2018-01,2018,1,1,510,510,510,2018-01-01,0
160201,2018-01-02,7,1600002,510,1.0,160,ギフト - 小型ゲーム,ギフト,2018-01,2018,1,2,510,510,510,2018-01-01,1
160202,2018-01-02,10,1600002,510,1.0,160,ギフト - 小型ゲーム,ギフト,2018-01,2018,1,2,510,510,510,2018-01-01,1
160203,2018-01-02,0,1600002,510,2.0,160,ギフト - 小型ゲーム,ギフト,2018-01,2018,1,2,510,510,510,2018-01-01,1
160204,2018-01-02,14,1600002,510,1.0,160,ギフト - 小型ゲーム,ギフト,2018-01,2018,1,2,510,510,510,2018-01-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119556,2019-10-30,16,2400100,1020,1.0,240,ギフト - ボードゲーム,ギフト,2019-10,2019,10,30,1020,1020,1020,2019-10-30,0
1119557,2019-10-31,5,2400100,1020,1.0,240,ギフト - ボードゲーム,ギフト,2019-10,2019,10,31,1020,1020,1020,2019-10-30,1
1119567,2019-10-31,10,1700230,950,1.0,170,ギフト - カード、パズル等,ギフト,2019-10,2019,10,31,950,950,950,2019-10-31,0
1119568,2019-10-31,4,2100176,2280,1.0,210,ギフト - ぬいぐるみ,ギフト,2019-10,2019,10,31,2280,2280,2280,2019-10-31,0


<p>商品カテゴリー大分類はlabel encordingで処理。</p>

In [217]:
le = LabelEncoder()

le.fit(train["商品カテゴリ名_大分類"])

train["商品カテゴリ名_大分類"] = le.transform(train["商品カテゴリ名_大分類"])

le.fit(test["商品カテゴリ名_大分類"])

test["商品カテゴリ名_大分類"] = le.transform(test["商品カテゴリ名_大分類"])

In [218]:
tmp = df_sales_history[(df_sales_history["年"]==2018) & (df_sales_history["月"]<12)].groupby(["年月", "商品ID", "店舗ID"], as_index=False)["売上個数"].sum()

avg_sales_2018_normal_month = tmp.groupby(["商品ID", "店舗ID"], as_index=False)["売上個数"].mean()

In [219]:
train_before = train.copy()

def month_alocate(row):
    years = row["年"].values
    months = row["月"].values
    new_years = []
    new_months = []
    
    for year ,month in zip(years,months):
        if month == 11:
            new_years.append(2019)
            new_months.append(1)
            
        elif month == 12:
            new_years.append(2019)
            new_months.append(2)
        
        else:
            new_years.append(year)
            new_months.append(month+2)
            
    return new_years, new_months

train_before["年"], train_before["月"] = month_alocate(train_before)

In [220]:
train_before = train_before.rename(columns={"売上個数":"sale_last_month", "item_price_min":"item_price_last_month"})

print(f"マージ前{len(train)}")
train = pd.merge(train, train_before[["年", "月", "店舗ID", "商品ID", "商品カテゴリID", "商品カテゴリ名_大分類","sale_last_month","item_price_last_month"]],
                 on=["年", "月", "店舗ID", "商品ID", "商品カテゴリID", "商品カテゴリ名_大分類"], how="left")
print(f"マージ後{len(train)}")

print(f"マージ前{len(test)}")
test = pd.merge(test, train_before[["年", "月", "店舗ID", "商品ID", "商品カテゴリID", "商品カテゴリ名_大分類","sale_last_month","item_price_last_month"]], 
                on=["年", "月", "店舗ID", "商品ID", "商品カテゴリID", "商品カテゴリ名_大分類"], how="left")
print(f"マージ後{len(test)}")

train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

マージ前495134
マージ後495134
マージ前3060
マージ後3060


In [221]:
train["item_price_difference"] = train["item_price_max"] - train["item_price_min"]
test["item_price_difference"] = test["item_price_max"] - test["item_price_min"]

train["item_price"] = train["item_price_mean"]
test["item_price"] = test["item_price_min"]

train.drop("item_price_min", axis=1 ,inplace=True)
test.drop("item_price_min", axis=1 ,inplace=True)
train.drop("item_price_max", axis=1 ,inplace=True)
test.drop("item_price_max", axis=1 ,inplace=True)
train.drop("item_price_mean", axis=1 ,inplace=True)
test.drop("item_price_mean", axis=1 ,inplace=True)

In [222]:
train

,年,月,店舗ID,商品ID,商品カテゴリID,商品カテゴリ名_大分類,発売後経過期間,売上個数,sale_last_month,item_price_last_month,item_price_difference,item_price
0,2018,1,0,1000001,100,5,12.600,6.0,0.0,0.0,0,420.0
1,2018,1,0,1000002,100,5,10.000,2.0,0.0,0.0,0,250.0
2,2018,1,0,1000004,100,5,15.000,1.0,0.0,0.0,0,120.0
3,2018,1,0,1000005,100,5,11.875,8.0,0.0,0.0,120,220.0
4,2018,1,0,1000006,100,5,4.000,2.0,0.0,0.0,120,190.0
...,...,...,...,...,...,...,...,...,...,...,...,...
495129,2019,10,17,3400005,340,4,661.000,1.0,4.0,1190.0,0,1190.0
495130,2019,10,17,3400006,340,4,642.000,1.0,1.0,2700.0,0,2700.0
495131,2019,10,17,3400012,340,4,639.000,1.0,2.0,680.0,0,680.0
495132,2019,10,17,3400016,340,4,141.000,1.0,1.0,250.0,0,250.0


In [223]:
train = train.rename(columns={"年":"year", "月":"month", "店舗ID":"shopID", "商品ID":"itemID", 
                              "商品カテゴリID":"item_categoryID","商品カテゴリ名_大分類":"item_category_big",
                    "発売後経過期間":"release_time", "売上個数":"y"})

test = test.rename(columns={"年":"year", "月":"month", "店舗ID":"shopID", "商品ID":"itemID", 
                            "商品カテゴリID":"item_categoryID","商品カテゴリ名_大分類":"item_category_big",
                    "発売後経過期間":"release_time"})

test.drop("日付", axis=1 ,inplace=True)

In [224]:
train.to_pickle('../input/pickle/train.zip')
test.to_pickle('../input/pickle/test.zip')